## Import library

In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime

import yfinance as yf
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource

import sambo
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy

from backtesting.test import SMA, GOOG

# backtesting.set_bokeh_output(notebook=True)
            
import itertools 
import logging
logger = logging.getLogger('yfinance')
logger.disabled = True

C:\Users\ASUS\AppData\Local\Temp\ipykernel_17140\1650128870.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
d:\Binance Tradebot\.venv\Lib\site-packages\backtesting\_plotting.py:53: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

## Import utils

In [2]:
from utils.loader import *
from utils.signals import *
from utils.trade import *
from utils.strategy import *

## Plotting sample

In [3]:
# now = datetime.today().strftime('%Y-%m-%d')

loader = DataLoader(ticker='AAPL', start='2010-06-01', end='2020-12-31', freq='1d', test_size=0.2)
loader.run()
# test     

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


## Strategy evaluation 

In [4]:
# strategy = EmaCross
strategy = BollingerBound
bt = BackTrader(data=loader.data)

# params = {
#     'short_duration': 5,
#     'long_duration': 10
# }
params = {'take_profit_ratio': 1.15}
bt.evaluate(data=bt.test_data, strategy=strategy, params=params, order_size=0.9999, plot=True)
bt.trades.head()

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,ReturnPct,EntryTime,ExitTime,Duration,Tag,Entry_Bollinger…(C)_0,Exit_Bollinger…(C)_0,Entry_Bollinger…(C)_1,Exit_Bollinger…(C)_1,Entry_Bollinger…(C)_2,Exit_Bollinger…(C)_2
0,266,23,25,37.407196,35.336829,NaN,43.018275,-550.717518,-0.055347,2018-12-20,2018-12-24,4 days,Long,44.302869,45.020921,40.944576,40.330265,37.586283,35.639609
1,276,31,49,33.915245,39.002532,32.219483,39.002532,1404.091145,0.150000,2019-01-03,2019-01-30,27 days,Long,42.956487,38.848016,38.596817,36.625446,34.237146,34.402876
2,-272,49,83,39.415531,45.327861,NaN,37.444755,-1608.153671,-0.150000,2019-01-30,2019-03-20,49 days,Short,38.848016,45.388864,36.625446,42.650385,34.402876,39.911905
3,-194,84,87,46.732574,44.395946,53.742461,44.395946,453.305972,0.050000,2019-03-21,2019-03-26,5 days,Short,46.111329,46.828357,42.938196,43.477769,39.765064,40.127182
4,-188,112,118,50.428738,47.907301,57.993048,47.907301,474.030134,0.050000,2019-05-01,2019-05-09,8 days,Short,50.305364,50.799714,48.341103,48.911217,46.376841,47.022720


## Cross-validation sample

In [5]:
# grid_search = {
#     'short_duration': range(2, 4),
#     'long_duration': range(5, 11)
# }
grid_search={'take_profit_ratio': [1.05, 1.1], 'stop_loss_ratio': [0.95, 0.98]}

bt.cross_val(strategy=strategy, train_size=160, test_size=80, step_size=40, order_size=0.999, commission=0.002, grid=grid_search)

{'take_profit_ratio': 1.1, 'stop_loss_ratio': 0.9672324314346219}
8622.646457723526
{'take_profit_ratio': 1.0625486314381436, 'stop_loss_ratio': 0.98}
9350.61397024354
{'take_profit_ratio': 1.1, 'stop_loss_ratio': 0.9568118541275417}
10000.0
{'take_profit_ratio': 1.1, 'stop_loss_ratio': 0.9571318611553796}
10360.699631963182
{'take_profit_ratio': 1.1, 'stop_loss_ratio': 0.9571318611553796}
10000.0
{'take_profit_ratio': 1.1, 'stop_loss_ratio': 0.95}
9462.851478488159
{'take_profit_ratio': 1.0872559324803635, 'stop_loss_ratio': 0.9598544319008827}
10822.263643977478
{'take_profit_ratio': 1.1, 'stop_loss_ratio': 0.9672324314346219}
9109.282658552334
{'take_profit_ratio': 1.1, 'stop_loss_ratio': 0.9695599963549579}
10946.269809228517
{'take_profit_ratio': 1.0602087677815613, 'stop_loss_ratio': 0.9620053549655859}
9053.917764410044
{'take_profit_ratio': 1.1, 'stop_loss_ratio': 0.9646661228266882}
9444.251149298781
{'take_profit_ratio': 1.058867172090887, 'stop_loss_ratio': 0.963394885275155

9840.162072010027

In [ ]:
class MacdCross(Strategy):

    id = 0
    take_profit_ratio = 1.02
    stop_loss_ratio = 0.95

    def init(self):
        price = self.data.Close
        self.macd_line = self.I(MACD, price)[0]
        self.signal_line = self.I(MACD, price)[1]

    def next(self):
        entry_price = self.data.Close[-1]

        if crossover(self.macd_line, self.signal_line):
            self.buy(size = self.order_size, tp=entry_price*self.take_profit_ratio, sl=entry_price*self.stop_loss_ratio, tag=f'Long {self.id}')
            self.id += 1

        elif crossover(self.signal_line, self.macd_line):
            for trade in self.trades:
                if trade.tag ==f'Long {self.id-1}':
                    trade.close()